# fMRI preprocessing
This tutorial shows how to run the NIAK fMRI preprocessing pipeline, using a limited set of options. See the [documentation](http://niak.simexp-lab.org/pipe_preprocessing.html) of the pipeline for a more comprehensive list of options. Download the tutorial as a notebook [here](https://raw.githubusercontent.com/SIMEXP/niak_tutorials/master/fmri_preprocessing/niak_tutorial_fmri_preprocessing.ipynb) and a matlab script [here](https://raw.githubusercontent.com/SIMEXP/niak_tutorials/master/fmri_preprocessing/niak_tutorial_fmri_preprocessing.m). We recommend to use [jupyter](http://jupyter.org/) from a niak docker container, as described in the [NIAK installation page](http://niak.simexp-lab.org/niak_installation.html). 

## Preparing files
First download a small fMRI dataset, with a structural scan. Be aware that all raw and derivatives data will be generated in the current folder. Note that you will need to manually remove the `data_test_niak_mnc1` and `fmri_preprocess` folders to restart this tutorial from scratch.

In [1]:
clear
niak_wget('data_test_niak_mnc1');

Now, set up the names of the structural and functional files.

In [2]:
path_data = [pwd filesep];
% Structural scan
files_in.subject1.anat = ...
    [path_data 'data_test_niak_mnc1/anat_subject1.mnc.gz'];       
% fMRI run 1
files_in.subject1.fmri.session1.motor = ...
    [path_data 'data_test_niak_mnc1/func_motor_subject1.mnc.gz'];

We start by specifying where to write the results.

In [3]:
% Where to store the results
opt.folder_out  = [path_data 'fmri_preprocess/'];

## Parallel computing 
Next we specify how many threads to use. A value of `N` means that up to `N` jobs can be executed in parallel. 

In [4]:
% Use up to 2 threads
opt.psom.max_queued = 2;

## Slice timing
We now set the options of the slice timing correction. Note that we specify the type of the scanner (in practice, only 'Siemens' has an impact), because the definition of the scanner impacts the definition of the slice timing.

In [5]:
opt.slice_timing.type_acquisition = 'interleaved ascending'; 
opt.slice_timing.type_scanner     = 'Bruker';                
opt.slice_timing.delay_in_tr      = 0;

other options are available as part of the slice timing step, which are not part of the slice timing per say. It is notably possible to center the functional images (if the header information is not accurate), or to suppress some volumes at the beginning of the time series: 

In [6]:
% Center the functional volumes on the brain center-of-mass (true/false)
opt.slice_timing.flag_center = false;
% Suppress some volumes at the beginning of the run
opt.slice_timing.suppress_vol = 3;

## Resampling
The voxel size for resampling in stereotaxic space is specified by the user:

In [7]:
% The voxel size to use in the stereotaxic space
opt.resample_vol.voxel_size    = 10;

## T1 processing
The options for non-uniformity correction of the T1 image is often useful to tweak:

In [8]:
% Parameter for non-uniformity correction. 
% 200 is a suggested value for 1.5T images, 
% 75 for 3T images. 
opt.t1_preprocess.nu_correct.arg = '-distance 75';

## Regression of confounds
The options for temporal filtering, motion parameter regression, white matter and ventricle signal regression, COMPCOR, global signal correction and scrubbing.

In [9]:
% Cut-off frequency for high-pass filtering, 
% or removal of low frequencies (in Hz). 
opt.time_filter.hp = 0.01; 
% Cut-off frequency for low-pass filtering, 
% or removal of high frequencies (in Hz). 
opt.time_filter.lp = 0.1;
% Remove slow time drifts (true/false)
opt.regress_confounds.flag_slow = true;
% Remove high frequencies (true/false)
opt.regress_confounds.flag_high = false;
% Apply regression of motion parameters (true/false)
opt.regress_confounds.flag_motion_params = true;
% Reduce the dimensionality of motion parameters with PCA (true/false)
opt.regress_confounds.flag_pca_motion = true;
% How much variance of motion parameters (with squares) to retain
opt.regress_confounds.pct_var_explained = 0.95;
% Apply average white matter signal regression (true/false)         
opt.regress_confounds.flag_wm = true;
% Apply average ventricle signal regression (true/false)         
opt.regress_confounds.flag_vent = true;
% Apply anat COMPCOR (white matter+ventricles, true/false)
% We recommend not using FLAG_WM and FLAG_VENT together with FLAG_COMPCOR
opt.regress_confounds.flag_compcor = false;
% Apply global signal regression (true/false)         
opt.regress_confounds.flag_gsc = true; 
% Apply scrubbing (true/false)
opt.regress_confounds.flag_scrubbing = true;     
% The threshold on frame displacement for scrubbing 
opt.regress_confounds.thre_fd = 0.5;    

## Spatial smoothing
The size of the spatial Gaussian blurring kernel: 

In [10]:
% Full-width at maximum (FWHM) of the Gaussian blurring kernel, in mm.
opt.smooth_vol.fwhm      = 6;

## Run the pipeline
This command can take up to 20-30 minutes to complete. It runs the full preprocessing pipeline on one subject/run. 

In [ ]:
niak_pipeline_fmri_preprocess(files_in,opt);
% Check the content of fmri_preprocess/logs/PIPE_history.txt to monitor the progress of the pipeline

Generating pipeline for individual fMRI preprocessing :
    Adding subject1 ; 0.25 sec
Adding group-level quality control of coregistration in anatomical space (linear stereotaxic space) ; 0.01 sec
Adding group-level quality control of coregistration in anatomical space (non-linear stereotaxic space) ; 0.01 sec
Adding group-level quality control of coregistration in functional space ; 0.02 sec
Adding group-level quality control of motion correction (motion parameters) ; 0.02 sec
Adding group-level quality control of scrubbing time frames with excessive motion ; 0.00 sec
Adding the report on fMRI preprocessing ; 0.24 sec

Logs will be stored in /sandbox/home/git/niak_tutorials/fmri_preprocessing/fmri_preprocess/logs/
Generating dependencies ...
   Percentage completed :  0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100- 0.05 sec
Setting up the to-do list ...
   I found 50 job(s) to do.
I could not find any log file. This pipeline has not been started (yet?). Press CTRL-C to

Once the pipeline has completed, an interactive report is built as part of the output. Just open the file `fmri_preprocess/report/index.html` in your browser. Note that because of the very low resolution in the functional images of these data, the T1-BOLD registration fails. Check an example report on a real sample [here](https://simexp.github.io/qc_cobre/index.html).  